In [ ]:
# Code from https://github.com/parth647/reddit_scraping_using_praw/blob/master/python_reddit_scrapy.py
# Code from https://github.com/D3vd/Reddit_Image_Scraper/blob/master/Reddit_image_scraper.py

In [ ]:
import json
import re
import praw
from pmaw import PushshiftAPI #to allow to mine more than 1000 records

import os
import requests
import pandas as pd
import configparser
from prawcore.exceptions import ResponseException

from reddit_image_scrapper import get_img_subreddit, delete_img_list

In [ ]:
def get_client_info():
  config = configparser.ConfigParser()
  config.read("config.ini")
  id = config["ALPHA"]["client_id"]
  secret = config["ALPHA"]["client_secret"]

  return id, secret

In [ ]:
# Acessing the reddit api

clien_id, client_secret = get_client_info()
reddit = praw.Reddit(client_id=clien_id,#my client id
                     client_secret=client_secret,  #your client secret
                     user_agent='retrieve kb data', #user agent name
                     username = '',     # your reddit username
                     password = '')     # your reddit password

sub = ['MechanicalKeyboards']  # make a list of subreddits you want to scrape the data from
# Alternate sub to mine: 'MechanicalKeyboards'

## To scrape stickied comments and pictures of the reddit post
main file -> json with the following format
{'sticky_comment': raw_text, 'title': raw_text, 'img_path': [path_to_saved_images]}

In [ ]:
list_scraped_dict = []

delete_img_list() # Delete image list to get clean slate
os.makedirs('data', exist_ok=True)

api = PushshiftAPI(praw=reddit)

try:
  #for i, submission in enumerate(reddit.subreddit('MechGroupBuys').new(limit=2000)):    
  for i, submission in enumerate(api.search_submissions(subreddit='MechGroupBuys', limit=3000)):    
    print(i, submission.title)
    title_pattern = '\[(?:GB|Pre-order|Preorder|In-stock)\](.*)/' #Bring text eda code to help save files
    try:
      title = re.findall(title_pattern, submission.title.replace('\n', ''))[0].replace('/', '').strip()
    except:
      title = submission.title.replace('[GB]', '').strip()
      title = title[:title.find('//')]


    if os.path.exists(os.path.join('data', f'scrapped_data_{title}.json')): continue
    print(submission.title)
    for top_level_comment in submission.comments:
      if top_level_comment.stickied:        

        # If the key is to extract the price, try to parse all rows into text file then manipulate from there
        sticky_comment = top_level_comment.body

    list_file_loc = get_img_subreddit(submission, num_img_limit=50, sleep_interval=1)
    scraped_dict = {'title': submission.title, 'sticky_comment': sticky_comment, 'img_paths': list_file_loc}

    with open(os.path.join('data', f'scrapped_data_{title}.json'), 'w') as json_output:
      json.dump(scraped_dict, json_output, indent=2)

    #list_scraped_dict.append(scraped_dict)
      
  #with open("scrapped_data.json", "w") as json_output:
   #json.dump(list_scraped_dict, json_output, indent=2)
  
          
except ResponseException as e:
  pass